# Parameter definition
Choose all the settings for the database creation.

In [1]:
db_name= 'test_video_dataset' #'colab_main_dataset'
input_dir = '/home/andrea/Desktop/image_datasets'
output_dir = '/home/andrea/Desktop/sql_database'
database_names=['lfw'] # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["ArcFace"] # , "Dlib", "ArcFace",
attributes_to_update=['gender', 'age']  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser-fiq'] # 'confusion_score',
save_in_drive = False # To work in google colab

# Database creation

database = SQLDataBase(db_name=db_name,
                        database_names=database_names, # , 'scface', 'forenface', 'enfsi', 'enfsi2015'
                        detector_names=detector_names,
                        embedding_model_names=embedding_model_names,
                        quality_model_names=quality_model_names,
                        save_in_drive=save_in_drive                        
                        )

database.fill_db()
database.update_db(attributes_to_update=attributes_to_update, force_update=False)

In [2]:
from sql_face.sqldb import SQLDataBase

In [3]:
database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

In [4]:
database.__dict__

{'db_name': 'test_video_dataset',
 'input_dir': '/home/andrea/Desktop/image_datasets',
 'save_in_drive': False,
 'output_dir': '/home/andrea/Desktop/sql_database',
 'session': <sqlalchemy.orm.session.Session at 0x7f39b2b63450>,
 'databases': [<sql_face.databases.LFW at 0x7f391ce13e50>],
 'detector_names': ['mtcnn'],
 'embedding_model_names': ['ArcFace'],
 'quality_model_names': ['ser-fiq']}

In [5]:
database.create_tables()

Creating image record from LFW: 0it [00:00, ?it/s]
TRIM Creating Cropped Images for detector mtcnn:   0%|          | 0/5 [00:00<?, ?it/s]

1/1 [==============================] - 0s 147ms/step


TRIM Creating Cropped Images for detector mtcnn:  20%|██        | 1/5 [00:01<00:07,  1.95s/it]

1/1 [==============================] - 0s 66ms/step


TRIM Creating Cropped Images for detector mtcnn:  40%|████      | 2/5 [00:03<00:04,  1.59s/it]

...

TRIM Creating Cropped Images for detector mtcnn:  60%|██████    | 3/5 [00:04<00:03,  1.51s/it]

1/1 [==============================] - 0s 52ms/step


TRIM Creating Cropped Images for detector mtcnn:  80%|████████  | 4/5 [00:06<00:01,  1.43s/it]

1/1 [==============================] - 0s 83ms/step


Quality models: 100%|██████████| 1/1 [00:00<00:00, 52.12it/s]


In [6]:
database.update_tables(attributes_to_update)

Updating attributes in databases:   0%|          | 0/1 [00:00<?, ?it/s]

Database <sql_face.databases.LFW object at 0x7f391ce13e50>


1/1 [==============================] - 0s 494ms/step


1/1 [==============================] - 0s 451ms/step


1/1 [==============================] - 0s 455ms/step


...

1/1 [==============================] - 1s 682ms/step


1/1 [==============================] - 1s 833ms/step


1/1 [==============================] - 1s 545ms/step


1/1 [==============================] - 1s 701ms/step


1/1 [==============================] - 1s 755ms/step
